In [7]:
using Polyhedra, CDDLib
using DrakeVisualizer
import ColorTypes: RGBA

In [83]:
# make a simple polyhedron

function polyhedron_from_bounds(args...)
    dim = length(args)
    A = zeros(Float64, 2*dim, dim)
    b = zeros(Float64, 2*dim)
    
    for i=1:dim
        bounds = args[dim]
        idx_lb = 2*(i-1)+1
        idx_ub = 2*(i-1)+2
        
        A[idx_lb, i] = -1
        b[idx_lb] = -bounds[1]
        
        A[idx_ub, i] = 1
        b[idx_ub] = bounds[2]
    end
    
    poly = polyhedron(SimpleHRepresentation(A,b))
    return poly
end

# converts polynomial in H representation to 3d by introducing -1,1 bounds on y vars
function convert_polyhedron_to_3d(poly)
    hRep = SimpleHRepresentation(poly)
    dim = size(hRep.A)[2]

    
    #if dimension already 3 we are done, just return poly
    if !(dim in [2,3])
        error("dimension must be 2 or 3")
    end
    
    if
        dim == 3
        return poly
    end
    
    # assume it is specified in (x,z) space, lift it to x,y,z space
    A = hRep.A
    b = hRep.b
    A_size = size(A)
    ybounds = [-1,1]
    
    A_3d = zeros(Float64, A_size[1]+2, 3)
    A_3d[1:A_size[1],1] = A[:,1] # x coordinate
    A_3d[1:A_size[1],3] = A[:,2] # z coordinate
    A_3d[end-1, 2] = -1
    A_3d[end, 2] = 1
    
    b_3d = zeros(Float64, length(b) + 2)
    b_3d[1:length(b)] = b
    b_3d[end-1] = -ybounds[1]
    b_3d[end] = ybounds[2]
    
    poly_3d = polyhedron(SimpleHRepresentation(A_3d, b_3d))
    return poly_3d    
end

p = polyhedron_from_bounds([-1,1], [-1,1], [-1,1])

# visualize that polyhedron in DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
red = RGBA(1.,0.,0.)
geom_data = GeometryData(p, red)
vis = Visualizer()

Visualizer with path prefix Symbol[] using LCM LCMCore.LCM(Ptr{Void} @0x00000000050f07c0,"",RawFD(174),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#8{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#8{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x0000000006799a10)])

In [84]:
setgeometry!(vis[:poly], geom_data)

Visualizer with path prefix Symbol[:poly] using LCM LCMCore.LCM(Ptr{Void} @0x00000000050f07c0,"",RawFD(174),LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#8{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#8{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT,DrakeVisualizer.handle_msg),Ptr{Void} @0x0000000006799a10)])

In [90]:
p = polyhedron_from_bounds([-1,1], [-2,2],)
p_3d = convert_polyhedron_to_3d(p)

hRep = SimpleHRepresentation(p_3d)
println(hRep.A)
println(hRep.b)
setgeometry!(vis[:poly], GeometryData(p_3d, red))
typeof(p)

[-1.0 0.0 0.0; 1.0 0.0 0.0; 0.0 0.0 -1.0; 0.0 0.0 1.0; 0.0 -1.0 0.0; 0.0 1.0 0.0]
[2.0,2.0,2.0,2.0,1.0,1.0]


CDDLib.CDDPolyhedron{2,Float64}

In [69]:
a = zeros(3,3)
a[end-1,2] = 1
b = zeros(10)
length(b)

10

In [92]:
import Polyhedra

In [93]:
Polyhedra.Polyhedron

Polyhedra.Polyhedron{N,T}